In [1]:
from library1 import *

### Initial Setup

In [2]:
x_train, y_train, x_valid, y_valid = get_data()
train_ds, valid_ds = Dataset(x_train, y_train), Dataset(x_valid, y_valid)
nh, bs = 50, 512
c = y_train.max().item()+1
loss_func = F.cross_entropy

In [3]:
data = DataBunch(*get_dls(train_ds, valid_ds, bs), c)

In [4]:
def create_learner(model_func, loss_func, data):
    return Learner(*model_func(data), loss_func, data)

In [7]:
learn = create_learner(get_model, loss_func, data)
run = Runner([AvgStatsCallback([accuracy])])

run.fit(10, learn)

train: [0.659440546875, tensor(0.8112)]
valid: [0.3114064453125, tensor(0.9109)]
train: [0.29089189453125, tensor(0.9156)]
valid: [0.24649462890625, tensor(0.9311)]
train: [0.229677265625, tensor(0.9332)]
valid: [0.20318719482421874, tensor(0.9420)]
train: [0.1960532421875, tensor(0.9434)]
valid: [0.17485281982421874, tensor(0.9516)]
train: [0.1721002734375, tensor(0.9493)]
valid: [0.1686032958984375, tensor(0.9541)]
train: [0.15405513671875, tensor(0.9554)]
valid: [0.15477152099609376, tensor(0.9584)]
train: [0.14082705078125, tensor(0.9593)]
valid: [0.1439829833984375, tensor(0.9606)]
train: [0.128323251953125, tensor(0.9630)]
valid: [0.13447869873046875, tensor(0.9633)]
train: [0.119104580078125, tensor(0.9663)]
valid: [0.1368278076171875, tensor(0.9600)]
train: [0.111301064453125, tensor(0.9683)]
valid: [0.12423985595703126, tensor(0.9657)]


In [8]:
def get_model_func(lr=0.5): return partial(get_model, lr=lr)

### Annealing

In [9]:
class Recorder(Callback):
    def begin_fit(self): self.lrs,self.losses = [],[]

    def after_batch(self):
        if not self.in_train: return
        self.lrs.append(self.opt.param_groups[-1]['lr'])
        self.losses.append(self.loss.detach().cpu())        

    def plot_lr  (self): plt.plot(self.lrs)
    def plot_loss(self): plt.plot(self.losses)

class ParamScheduler(Callback):
    _order=1
    def __init__(self, pname, sched_func): self.pname,self.sched_func = pname,sched_func

    def set_param(self):
        for pg in self.opt.param_groups:
            pg[self.pname] = self.sched_func(self.n_epochs/self.epochs)
            
    def begin_batch(self): 
        if self.in_train: self.set_param()

In [10]:
def sched_lin(start, end):
    def _inner(start, end, pos): return start + pos*(end-start)
    return partial(_inner, start, end)

In [11]:
def annealer(f):
    def _inner(start, end): return partial(f, start, end)
    return _inner

@annealer
def sched_lin(start, end, pos): return start + pos*(end-start)

In [ ]:
sched_lin()